# Лабораторная работа №3
## Выполнил студент группы Станолевич Владислав БВТ2004

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [9]:
import time
def prefix_function(text):
    n = len(text)
    pi = [0]*n
    for i in range(1, n):
        j = pi[i-1]
        while j > 0 and text[j] != text [i]:
            j = pi[j-1]
        if text[i] == text[j]:
            j += 1
        pi[i] = j
    return pi

def kmp_search(text, sub_text, start_index=0):
    j = 0
    pi = prefix_function(sub_text)
    for i in range(start_index, len(text)):
        while j > 0 and text[i] != sub_text[j]:
            j = pi[j-1]
        if text[i] == sub_text[j]:
            j+=1
        if j >= len(sub_text):
            return i-j+1
    return None

text = input('Введите строку: ')
sub_text = input('Введите подстроку: ')
f=input('Если Вам нужна чувствительность регистра введите - 1, если нет - 0: ')
if(f=="0"):
    text=text.lower()
    sub_text=sub_text.lower()
start_time = time.perf_counter()
print(kmp_search(text, sub_text, 0))
print("Время Алгоритма Кнута-Морриса-Пратта: --- {:.6f} ms ---".format((time.perf_counter() - start_time) * 1000))

start_time = time.perf_counter()
print(text.find(sub_text))
print("Результат стандартного поиска: --- {:.6f} ms ---".format((time.perf_counter() - start_time) * 1000))


Введите строку: dfgseqwertyfbndfl
Введите подстроку: qwerty
Если Вам нужна чувствительность регистра введите - 1, если нет - 0: 0
5
Время Алгоритма Кнута-Морриса-Пратта: --- 0.215200 ms ---
5
Результат стандартного поиска: --- 0.104700 ms ---


#### Упрощенный алгоритм Бойера-Мура

In [10]:
import time
def bmPredCompil(x):
    d = {}
    lenX = len(x)
    for i in range(len(x)):
        # сколько символов с правого края до этой буквы
        d[ord(x[i])] = lenX - i
    return d


def boyerMurSearch(s, x):
    d = bmPredCompil(x)
    # k - проход по s
    # j - проход по x
    # i - место начала прохода по s
    lenX = i = j = k = len(x)
    while j > 0 and i<=len(s):
     # совпали, двигаемся дальше (от конца к началу)
        if s[k-1] == x[j-1]:
            k -= 1
            j -= 1
        # иначе, продвигаемся по строке на d и начинаем с правого конца подстроки снова
        else:
            i += d[ord(s[i])]
            j = lenX
            k = i
    if j <= 0:# нашли
        return k
    return None # не нашли

text = input('Введите строку: ')
sub_text = input('Введите подстроку: ')
f=input('Если Вам нужна чувствительность регистра введите - 1, если нет - 0: ')
if(f=="0"):
    text=text.lower()
    sub_text=sub_text.lower()
start_time = time.perf_counter()
print(boyerMurSearch(text, sub_text))
print("Время Алгоритма Бойера-Мура: --- {:.6f} ms ---".format((time.perf_counter() - start_time) * 1000))

start_time = time.perf_counter()
print(text.find(sub_text))
print("Результат стандартного поиска: --- {:.6f} ms ---".format((time.perf_counter() - start_time) * 1000))

Введите строку: dfgseqwertyfbndfl
Введите подстроку: qwerty
Если Вам нужна чувствительность регистра введите - 1, если нет - 0: 0
5
Время Алгоритма Бойера-Мура: --- 0.245200 ms ---
5
Результат стандартного поиска: --- 0.111900 ms ---


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [1]:
import math
from queue import PriorityQueue

N = 4 #размерность игры
SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

#проверка решаемости позиции
def reshenie(mass): 
    summ=int(0)
    for i in range(len(mass)):
        for j in range(i, len(mass)):
            if mass[i]==0:
                summ=summ+math.floor((i+1)/4)
            elif mass[i]>mass[j]:
                summ += 1
    return summ%2==1

def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)     # вниз
    if i < N - 1: offsets.append(N)  # вверх
    if j > 0: offsets.append(-1)     # вправо
    if j < N - 1: offsets.append(1)  # влево
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))

class Position:
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    def __lt__(self, other):
        return self.start_distance < other.start_distance

if reshenie(start) == 0:
    print('Нерешаемая позиция')
else:
    # картеж
    start = tuple(start)
    p = Position(start, 0)
    # очередь 
    candidates = PriorityQueue()
    candidates.put(p)
    # множество 
    visited = set([p])

    came_from = {p.position: None}
    
    while p.position != SOLVED:
        p = candidates.get()
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)
    path = []
    prev = p
    while p.position != start:
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

[15, 14, 10, 13, 9, 10, 14, 15]


### Вывод